# 01_preprocessing_v1.2.1.ipynb 파일 설명

이 노트북은 **서울시 상권 분석**을 위한 데이터 전처리 및 통합 과정을 단계별로 수행하는 Jupyter Notebook입니다.
최종 데이터 프레임은 df_final의 이름으로 저장됩니다. 

---

## 주요 목적
- 여러 원천 데이터(상가, 인구, 지하철, 건물가격 등)를 하나의 분석용 데이터프레임으로 결합
- 각 상가(또는 점포)별로 다양한 공간적·상권적 특성(예: 근처 상점/음식점/카페 수, 인구, 지하철 접근성, 건물 시세 등)을 계산하여 추가

---

## 주요 처리 단계

- **데이터 로딩**
   - 상가, 인구, 지하철, 건물가격 등 다양한 csv 파일을 불러옴

- **데이터 전처리**
   - 필요 컬럼만 추출, 인덱스 설정 등

- **공간적 특성 계산**
   - BallTree, Haversine 등 거리 계산 알고리즘을 활용해  
     - 각 상가 주변 700m 이내 상점/음식점/카페 개수 계산  
     - 가장 가까운 지하철역 및 거리 계산
     - 각 상가별로 가장 가까운 지하철역의 승하차 인원 등 정보 결합

- **인구 데이터 결합**
   - 행정동코드를 기준으로 인구 데이터 결합

- **건물가격 데이터 결합**
   - 각 상가와 가장 가까운 건물의 평당 거래금액 정보 결합

- **최종 데이터 저장**
   - 결합된 데이터프레임을 csv로 저장

## 0. 데이터 로드

In [64]:
# python --v 3.13.3
import re
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import requests

import folium
from folium.plugins import HeatMap
import geopandas as gpd
import contextily as ctx

from dotenv import load_dotenv
import os
import math
from haversine import haversine
from sklearn.neighbors import BallTree

from distance_utils import DistanceCalculator, quick_distance, batch_distance_calculation


In [65]:
df_starbucks = pd.read_csv('../processed/df_starbucks.csv', encoding='utf-8-sig')
df_seoul = pd.read_csv('../processed/df_seoul.csv', encoding='utf-8-sig')
df_population = pd.read_csv('../processed/df_population.csv', encoding='utf-8-sig')
df_building_price = pd.read_csv('../processed/df_building_price.csv', encoding='utf-8-sig')
df_subway = pd.read_csv('../processed/df_subway.csv', encoding='utf-8-sig')
df_subway_on = pd.read_csv('../processed/df_subway_on.csv', encoding='utf-8-sig')
df_subway_off = pd.read_csv('../processed/df_subway_off.csv', encoding='utf-8-sig')

## 1. 데이터 결합: df_seoul, df_starbucks
- 방식: concat 
- df_seoul과 df_starbucks를 결합하기 위해 컬럼명, 형태 등을 맞춰줌.
    - (1) 행정동코드: df_starbucks는 10자리, df_seoul은 8자리 형태. --> df_seoul에 오른쪽에 00으로 패딩
    - (2) 시군구명: df_starbucks에 시군구명 추가
    - (3) 상호명: df_starbucks 상호명 앞에 '스타벅스'를 추가 변경
    - (4) 상가업소번호: df_starbucks 상가업소번호 수정 'STB'로 시작하는 형태


In [158]:
print(df_seoul.shape)
print(df_starbucks.shape)

(540517, 13)
(644, 13)


In [ ]:
# df_seoul 행정동코드가 8자리, df_starbucks 행정동코드가 10자리
df_seoul['행정동코드'] = df_seoul['행정동코드'].astype('str') + str('00')
# df.loc[df['스타벅스'] == 1, '행정동코드'] = df.loc[df['스타벅스'] == 1, '행정동코드'].apply(lambda x: x[:10])

In [68]:
# df_starbucks에 시군구명 추가, 상호명 앞에 스타벅스 추가변경, 상가업소번호 추가변경

# df_starbucks 시군구명 추가
df_starbucks['시군구명'] = df_starbucks['도로명주소'].str.extract(r'^서울.*\s([가-힣].*구)\s.*')

# df_starbucks 상호명 변경
df_starbucks['상호명'] = '스타벅스 ' + df_starbucks['상호명']

# df_starbucks 상가업소번호 변경
starbucks_id = [f'STB{i:03d}' for i in range(len(df_starbucks))]
df_starbucks['상가업소번호'] = starbucks_id

# df_seoul 데이터에 df_starbcuks 데이터 추가
df = pd.concat([df_seoul, df_starbucks], axis=0)
print(df.shape)
df.head(1).T


(541161, 13)


,0
상가업소번호,MA010120220800000033
상호명,부동산임대김은숙
상권업종대분류명,음식
상권업종중분류명,비알코올
상권업종소분류명,카페
시군구명,종로구
행정동코드,11110540
행정동명,삼청동
지번주소,서울특별시 종로구 삼청동 28-21
도로명주소,서울특별시 종로구 삼청로 122-1


In [69]:
df.isna().sum()

상가업소번호      0
상호명         0
상권업종대분류명    0
상권업종중분류명    0
상권업종소분류명    0
시군구명        0
행정동코드       0
행정동명        0
지번주소        0
도로명주소       0
경도          0
위도          0
스타벅스        0
dtype: int64

In [75]:
df = df.set_index('상가업소번호')

## 2. 공간 특성 추가
   - 방식: BallTree, Haversine 등 거리 계산 알고리즘 활용 --> distance_utils.py 참고
     - 각 상가 주변 700m 이내 상점/음식점/카페 개수 반환 
      - df_restaurant -> df '상권업종대분류명':'음식'
      - df_cafe -> df '상권업종대분류명':'카페'

In [70]:
distance_calc = DistanceCalculator()

In [71]:
df_restaurant = df[df['상권업종대분류명'] == '음식']
df_cafe = df[df['상권업종소분류명'] == '카페']

In [72]:
# 근처 상가, 식당, 카페 수 계산
df_with_features = distance_calc.calculate_all_nearby_features(
    df_base=df, 
    df_stores=df, 
    distance_km=0.7
)

In [ ]:
df = df_with_features.copy()

## 3. 데이터 결합: df, df_subway
- 방식: 공간 조인. 각 상가에서 가장 가까운 지하철역 반환 (위도, 경도 기반)

In [ ]:
# 가장 가까운 지하철 역, 거리
station_distance, station_name = distance_calc.find_closest_points(df, df_subway, '역명')

In [ ]:
df['근방지하철역'] = station_name
df['근방지하철역거리'] = station_distance

df.head(1).T

,0
상가업소번호,MA010120220800000033
상호명,부동산임대김은숙
상권업종대분류명,음식
상권업종중분류명,비알코올
상권업종소분류명,카페
시군구명,종로구
행정동코드,11110540
행정동명,삼청동
지번주소,서울특별시 종로구 삼청동 28-21
도로명주소,서울특별시 종로구 삼청로 122-1


In [76]:
df.isna().sum()

상호명         0
상권업종대분류명    0
상권업종중분류명    0
상권업종소분류명    0
시군구명        0
행정동코드       0
행정동명        0
지번주소        0
도로명주소       0
경도          0
위도          0
스타벅스        0
근방가게수       0
근방음식점수      0
근방카페수       0
근방지하철역      0
근방지하철역거리    0
dtype: int64

## 4. 데이터 결합: df_subway_on, df_subway_off
   - 방식: merge, inner join, key = 역번호

In [77]:
df_subway_on.head(1).T

,0
역명,언주
역번호,4126
호선명,9
승차_이용객수,10650.819672
승차_출근시간대,919.84918
승차_퇴근시간대,4441.531148


In [78]:
df_subway_ride = pd.merge(left=df_subway_on,
                          right=df_subway_off.drop(['역명','호선명'],axis=1),
                          how='inner',
                          on=['역번호'])
                        #   on=['역명','호선명'])
print(df_subway_on.shape)
print(df_subway_off.shape)
print(df_subway_ride.shape)

(286, 6)
(286, 6)
(286, 10)


In [79]:
# 호선이 여러개인 경우 역명 중복 출력 됨. 따라서 gropu by, sum으로 합계처리 해줌.
df_subway_ride = df_subway_ride.groupby('역명')[['승차_이용객수','승차_출근시간대','승차_퇴근시간대', '하차_이용객수', '하차_출근시간대','하차_퇴근시간대']].agg('sum').reset_index()
df_subway_ride.shape

(251, 7)

In [81]:
df_subway_ride['역명']  = df_subway_ride['역명'].str.split('(').str[0]

## 5. 데이터 결합: df, df_subway_ride
   - 방식: merge, inner join, key = 지하철역 이름 (df:근방지하철역, df_subway_ride:역명)

In [82]:
merged_df = pd.merge(left=df,
                        right=df_subway_ride,
                        how='inner',
                        left_on='근방지하철역',
                        right_on='역명')

In [83]:
print(merged_df.isna().sum())
merged_df.head(1).T

상호명         0
상권업종대분류명    0
상권업종중분류명    0
상권업종소분류명    0
시군구명        0
행정동코드       0
행정동명        0
지번주소        0
도로명주소       0
경도          0
위도          0
스타벅스        0
근방가게수       0
근방음식점수      0
근방카페수       0
근방지하철역      0
근방지하철역거리    0
역명          0
승차_이용객수     0
승차_출근시간대    0
승차_퇴근시간대    0
하차_이용객수     0
하차_출근시간대    0
하차_퇴근시간대    0
dtype: int64


,0
상호명,부동산임대김은숙
상권업종대분류명,음식
상권업종중분류명,비알코올
상권업종소분류명,카페
시군구명,종로구
행정동코드,11110540
행정동명,삼청동
지번주소,서울특별시 종로구 삼청동 28-21
도로명주소,서울특별시 종로구 삼청로 122-1
경도,126.98184


In [84]:
df = merged_df.copy()

## 6. 데이터 결합: df, df_population
   - 방식: merge, inner join, key = 행정동코드 (df:행정동코드, df_population:행정기관코드)

In [85]:
# 행정동 인구 merge: key = 행정동코드
df_population.head(1)

,행정기관코드,시도명,시군구명,읍면동명,계,0세_6세,7세_19세,20세_25세,26세_30세,31세_40세,41세_50세,51세_60세,61세_109세
0,1111051500,서울특별시,종로구,청운효자동,10978,309,1239,680,797,1486,1775,1849,2843


In [86]:
print(len(df['행정동코드'][0].astype('str')))
print(df['행정동코드'].dtype)
print(len(df_population['행정기관코드'][0].astype('str')))
print(df_population['행정기관코드'].dtype)

8
int64
10
int64


In [ ]:
# 행정동 인구 merge: key = 행정동코드 10자리. 

df_population['행정기관코드'] = df_population['행정기관코드'].astype('str')

df['행정동코드'] = df['행정동코드'].astype('str')

print(f"df_population:{df_population.shape}")
print(f"df:{df.shape}")


df_merged = pd.merge(left=df,
                     right=df_population.drop(columns=['시도명', '시군구명','읍면동명'], axis=1),
                     how='left',
                     left_on='행정동코드',
                     right_on='행정기관코드')

print(f"df_merged:{df_merged.shape}")

df_population:(3614, 13)
df:(518844, 24)
df_merged:(518844, 34)


In [118]:
df_merged.isna().sum()

상호명         0
상권업종대분류명    0
상권업종중분류명    0
상권업종소분류명    0
시군구명        0
행정동코드       0
행정동명        0
지번주소        0
도로명주소       0
경도          0
위도          0
스타벅스        0
근방가게수       0
근방음식점수      0
근방카페수       0
근방지하철역      0
근방지하철역거리    0
역명          0
승차_이용객수     0
승차_출근시간대    0
승차_퇴근시간대    0
하차_이용객수     0
하차_출근시간대    0
하차_퇴근시간대    0
행정기관코드      0
계           0
0세_6세       0
7세_19세      0
20세_25세     0
26세_30세     0
31세_40세     0
41세_50세     0
51세_60세     0
61세_109세    0
dtype: int64

In [119]:
df = df_merged.copy()

## 7. 데이터 결합: df, df_building_price
   - 방식: 공간조인. 각 상가에서 가장 가까운 상가의 가격 정보 붙이기

In [120]:
df_building_price.head(1)

,시군구,지번,도로명,용도지역,건축물주용도,전용/연면적(㎡),거래금액(만원),주소,경도,위도
0,서울특별시 동작구 노량진동,283-3,노량진로6길,제3종일반주거,제2종근린생활,9.88,"7,300",서울특별시 동작구 노량진동 283-3,126.936012,37.510849


In [121]:
df_building_price['거래금액(만원)'] = df_building_price['거래금액(만원)'].str.replace(',','')
df_building_price['거래금액(만원)'] = df_building_price['거래금액(만원)'].astype('int')
df_building_price['평당거래금액(만원)'] = df_building_price['거래금액(만원)'] / df_building_price['전용/연면적(㎡)'] / 3.3058
df_building_price['평당거래금액(만원)'] = df_building_price['평당거래금액(만원)'].round(1)
df_building_price.head(1)

,시군구,지번,도로명,용도지역,건축물주용도,전용/연면적(㎡),거래금액(만원),주소,경도,위도,평당거래금액(만원)
0,서울특별시 동작구 노량진동,283-3,노량진로6길,제3종일반주거,제2종근린생활,9.88,7300,서울특별시 동작구 노량진동 283-3,126.936012,37.510849,223.5


In [122]:
distance, price = distance_calc.find_closest_points(
    df1=df, 
    df2=df_building_price, 
    target_col='평당거래금액(만원)', 
    lat_col='위도', 
    lon_col='경도'
)


In [123]:
distance

array([0.20117832, 0.00203155, 0.02360837, ..., 0.26493835, 0.59106498,
       0.28215143], shape=(518844,))

In [124]:
df['평당거래금액(만원)'] = price.values
df.head(1).T

,0
상호명,부동산임대김은숙
상권업종대분류명,음식
상권업종중분류명,비알코올
상권업종소분류명,카페
시군구명,종로구
행정동코드,1111054000
행정동명,삼청동
지번주소,서울특별시 종로구 삼청동 28-21
도로명주소,서울특별시 종로구 삼청로 122-1
경도,126.98184


In [125]:
df.isna().sum()

상호명           0
상권업종대분류명      0
상권업종중분류명      0
상권업종소분류명      0
시군구명          0
행정동코드         0
행정동명          0
지번주소          0
도로명주소         0
경도            0
위도            0
스타벅스          0
근방가게수         0
근방음식점수        0
근방카페수         0
근방지하철역        0
근방지하철역거리      0
역명            0
승차_이용객수       0
승차_출근시간대      0
승차_퇴근시간대      0
하차_이용객수       0
하차_출근시간대      0
하차_퇴근시간대      0
행정기관코드        0
계             0
0세_6세         0
7세_19세        0
20세_25세       0
26세_30세       0
31세_40세       0
41세_50세       0
51세_60세       0
61세_109세      0
평당거래금액(만원)    0
dtype: int64

## 8. 데이터 정리
- 중복 특성 삭제: 행정기관코드, 역명
- 지하철역 인원 round 처리
- 최종 데이터 프레임: df_final

In [ ]:
df = df.drop(columns=['행정기관코드','역명'], axis=1)


In [148]:
col_ls = []
for col in df.columns:
    if re.findall(r'.*[승차|하차].*', col):
        col_ls.append(col)

col_ls.remove('근방지하철역')
col_ls.remove('근방지하철역거리')


col_ls.append('계')
col_ls

['승차_이용객수', '승차_출근시간대', '승차_퇴근시간대', '하차_이용객수', '하차_출근시간대', '하차_퇴근시간대', '계']

In [150]:
df[col_ls] = df[col_ls].round(0)

In [152]:
df['근방지하철역거리'] = df['근방지하철역거리'].round(2)

In [154]:
df.to_csv('../processed/df_final.csv', encoding='utf-8-sig', index=False)

In [155]:
df_final = pd.read_csv('../processed/df_final.csv', encoding='utf-8-sig')
df_final.head(1).T

,0
상호명,부동산임대김은숙
상권업종대분류명,음식
상권업종중분류명,비알코올
상권업종소분류명,카페
시군구명,종로구
행정동코드,1111054000
행정동명,삼청동
지번주소,서울특별시 종로구 삼청동 28-21
도로명주소,서울특별시 종로구 삼청로 122-1
경도,126.98184
